In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

/Users/anastasiakarpova/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1. Подключаемся к браузеру

In [ ]:
driver = webdriver.Safari()

2. Используя селениум, открываем сайт hh.ru

In [153]:
hh_url = 'https://hh.ru'
driver.get(hh_url)

3. Находим поисковую строку

In [ ]:
inp_tab = driver.find_element(By.ID, 'a11y-search-input')
inp_tab

4. Запускаем поиск - мы хотим изучить широкий спектр предложенных вакансий

In [155]:
inp_tab.send_keys(Keys.ENTER)
sleep(2)

5. Изучив html-код сайта, мы нашли все нужные нам элементы: ссылку, вакансию, зарплату, работодателя, город и опыт работы. 
Теперь создадим цикл, в котором будем собирать данные из каждого окошка с вакансией на странице и пройдем так по всем страницам (их всего 40)

In [ ]:
data = []

for p in range(39):  # для прохождения по всем 40 страницам
    hh = BeautifulSoup(driver.page_source, 'html.parser') # создаем "дерево" из html-кода
    ads = hh.find_all('div', {'class': 'vacancy-serp-item__layout'}) #находим все ячейки с описанием вакансии на странице
    
    for ad in ads: #проходимся по каждой ячейке
        try:
            link = ad.find('a', {'class': 'bloko-link'}).get('href')
            vacancy = ad.find('span', {'class': 'serp-item__title-link'}).text
            wage = ad.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            company = ad.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).find('span').text
            city = ad.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).text
            experience = ad.find('div', {'data-qa': 'vacancy-serp__vacancy-work-experience'}).text
            
        except:
            pass # на сайте иногда не указаны значения, например, зарплаты. пропускаем, чтобы код не падал
        data.append([link, vacancy, wage, company, city, experience])
    print(len(data))

    driver.find_element(By.XPATH, '//a[@data-qa="pager-next"]').click() # находим кнопку перехода на следующую страницу и тыкаем на нее (именно поэтому в цикл заходим 39 раз)
    sleep(10) # чтобы страница полность прогрузилась
        

6. Собранных данных оказалось недостаточно, поэтому в цикле ниже мы переходим по ссылке на страинцу отдельной вакансии и достаем оттуда еще пару признаков: график работы и рейтинг работодателя

In [ ]:
from fake_useragent import UserAgent
ua = UserAgent()
saf_ua = ua.safari
request_headers = {'user-agent': saf_ua} # сайт нас жестко банил, поэтому становимся более похожими на людей. 
# честно, скопировала эту штуку с сайта https://www.zenrows.com/blog/fake-user-agent#generate-fake-user-agents

extra_data = []
for vac in data: # проходимся по каждой отдельной вакансии
    vac_url = vac[0] # достаем ссылку
    r = requests.get(vac_url, headers= request_headers) 
    tree = BeautifulSoup(r.content, 'html.parser')
    try:
        employment = tree.find('p', {'data-qa': 'vacancy-view-employment-mode'}).text
        company_rate = tree.find('div', {'data-qa': 'employer-review-small-widget-total-rating'}).text
    except:
        pass # на сайте иногда не указаны значения, например, рейтинг работодателя. пропускаем, чтобы код не падал
    extra_data.append([vac_url, employment, company_rate])
    
    print(len(extra_data))
extra_data

7. Все готово! Теперь преобразуем полученные данные в датафрейм и на всякий случай делаем их копии.

In [157]:
df_1 = pd.DataFrame(data, columns = ['link', 'vacancy', 'wage', 'company', 'city', 'experience'])
df_1.shape


(2080, 6)

In [159]:
df_a = df_1
df_a.head()

,link,vacancy,wage,company,city,experience
0,https://adsrv.hh.ru/click?b=1003372&place=35&m...,Водитель со своим грузовым автомобилем,180 000 – 400 000 ₽,ИП Петрикин Александр Вячеславович,Москва,Без опыта
1,https://adsrv.hh.ru/click?b=993174&place=36&me...,Консультант поддержки в Яндекс Путешествия,от 30 000 ₽,Яндекс Крауд: Поддержка,Москва,Без опыта
2,https://hh.ru/vacancy/98672015?hhtmFrom=vacanc...,Менеджер по заполнению анкет (удаленно),36 800 – 60 000 ₽,ИП Кабин Дмитрий Владимирович,Москва,Без опыта
3,https://hh.ru/vacancy/98657055?hhtmFrom=vacanc...,Ручной тестировщик (QA Manual Junior),от 60 000 ₽,Oxytocin,Москва,Без опыта
4,https://hh.ru/vacancy/93848299?utm_source=head...,Торговый представитель (Водитель-курьер),170 000 – 250 000 ₽,ЦУМ,Москва,Опыт от 1 года до 3 лет


In [ ]:
extra_df = pd.DataFrame(extra_data, columns= ['link', 'employment', 'company_rate'])
extra_df.head()

In [162]:
df_b = extra_df
df_b.head()

,link,employment,company_rate
0,https://adsrv.hh.ru/click?b=1003372&place=35&m...,"Полная занятость, полный день","4,5"
1,https://adsrv.hh.ru/click?b=993174&place=36&me...,"Полная занятость, удаленная работа","4,0"
2,https://hh.ru/vacancy/98672015?hhtmFrom=vacanc...,"Полная занятость, удаленная работа","3,5"
3,https://hh.ru/vacancy/98657055?hhtmFrom=vacanc...,"Полная занятость, полный день","3,5"
4,https://hh.ru/vacancy/93848299?utm_source=head...,"Полная занятость, сменный график","4,0"


8. Сохраняем полученные таблицы в csv-файлы для удобства последующей работы

In [163]:
df_a.to_csv('df_a2.csv', encoding='utf-8')
df_b.to_csv('df_b2.csv', encoding='utf-8')
